GPS (Global Positioning System) is a technology that uses an array of GPS sattelites transmitting time and location signals to allow devices to accurately track location. This is done by collecting three signals from three different satellites. For each sattelite, the distance from the device to the sattelite is given by the speed of light times the time interval. Each such measurement places the position of the device on the surface of a sphere, and three spheres are needed to uniquely specify a location. If we assume some unknown amount of drift on the device's clock, we need a fourth to uniquely specify a point in space. To do this, we need to solve a system of nonlinear equations. Using curvature information and newton's method for optimizing allows us to accurate solve such systems.

(1) Solve the system by using Multivariate Newtons Method Find the receiver position (x, y, z) and time correction d for simultaneous satellite positions (Ai, Bi, Ci) equal to (15600, 7540, 20140), (18760, 2750, 18610), (17610, 14630, 13480), (19170, 610, 18390) km, and measured time intervals ti = 0.07074, 0.07220, 0.07690, 0.07242 sec, respectively. Initial vector (x0, y0, z0, d0) = (0, 0, 6370, 0).

To do this we need two tools: A way to solve systems of linear equations, and a way to take derivatives of multivariable functions. We will obtain both of these shortly.

In [ ]:
import numpy as np

#forward subsitution algorithm for lower triangular matrix L and vector b
def forward_sub(L,b):
  x = np.empty(len(b))

  #for each place value
  for i in range(len(b)):
    #make sure we're not dividing by zero
    if L[i,i] == 0:
      x[i] = 0
      continue

    #start off by setting the value to the corresponding value in b
    val = b[i]
    for j in range(i):
      #then subtract off the product of the row and the solution
      val = val - L[i,j]*x[j]
    #then divide off by the value on the diagonal
    val = val/L[i,i]

    #and set that x value to that value
    x[i] = val
  return x

def backward_sub(U,b):
  x = np.empty(len(b))

  for i in range(len(b)-1, -1, -1):
    #make sure we're not dividing by zero
    if U[i,i] == 0:
      x[i] = 0
      continue

    #start off by setting the value to the corresponding value in b
    val = b[i]
    for j in range(len(b)-1, i, -1):
      #then subtract off the product of the row and the solution
      val = val - U[i,j]*x[j]
    #then divide off by the value on the diagonal
    val = val/U[i,i]

    #and set that x value to that value
    x[i] = val
  return x

def PALU(A):
  n = len(A)
  P = np.identity(n)
  L = np.zeros((n,n))

  for i in range(n):
    pivot = i

    #find the biggest pivot, to avoid swamping
    for row in range(i+1,n):
      if (np.absolute(A[row,i]) > np.absolute(A[pivot,i])):
        pivot = row

    #if the whole column is 0 then we're already done
    if pivot == 0:
      continue

    #if the biggest pivot isn't the one on the diagonal then we need to swap rows
    if pivot != i:
      P[[pivot, i]] = P[[i,pivot]]
      A[[pivot, i]] = A[[i,pivot]]
      L[[pivot, i]] = L[[i,pivot]]

    #now we just subtract downwards
    pivot = A[i,i]
    pivrow = A[i]
    for row in range(i+1,n):
      lead = A[row,i]
      #if the leading coefficient is 0 we're already done
      if(lead == 0):
        continue

      coef = lead/pivot
      pivrow = coef*pivrow
      A[row] -= pivrow
      L[row,i] = coef

  #fill the diagonal of the lower triangular matrix with 1s
  for i in range(n):
    L[i,i] = 1

  return(P,L,A)

def palusolve(A,b):
  P,L,U = PALU(A)
  #permute b as required
  b = P.dot(b)
  #and then forward and backward sub to get our answer
  b = forward_sub(L,b)
  b = backward_sub(U,b)
  return b


A = np.matrix('-3.0 2 -5; 2 -3 4; 1 1 1')
b = np.array([-14, 10, 4])


A = np.array([[2.0, 1.0, 5.0],[4.0, 4.0, -4.0], [1.0, 3.0, 1.0]])
b = np.array([5,0,6])
print(palusolve(A,b))

A = np.array([[2.0,3.0],[3.0,2.0]])
b = np.array([4,1])
print(palusolve(A,b))

[-0.84210526  1.81578947  0.97368421]
[-1.  2.]


In [ ]:
def precond_conj_grad(matrix_A, guess_x, vector_b, iterations=30):
  r = vector_b - (matrix_A @ guess_x)
  D = np.diag(matrix_A)
  D_inverse = np.diag(1 / D)
  L = np.tril(matrix_A)
  U = np.triu(matrix_A)
  M = (D+L) @ D_inverse @ (D+U)
  M_inverse = np.linalg.inv(M)
  d = z = M_inverse @ r
  for k in range(iterations):
    if np.isclose(np.sum(r),0): return
    alpha = (r @ z)/(d @ matrix_A @ d)
    guess_x = guess_x + (alpha*d)
    r_prev = r
    r = r - alpha * (matrix_A @ d)
    z_prev = z
    z = (M_inverse @ r)
    beta = (r @ z)/(r_prev @ z_prev)
    d = z + (beta* d)
  return guess_x

In [ ]:
import time
PALU_time = []
conj_grad_time = []

A = np.array([[2.0, 1.0, 5.0, -2.0, 2.9],[4.0, 4.0, -4.0, 3.5, 7.1], [1.0, 3.0, 1.0, 2.6, -2.0], [7.0, 2.6, 1.0,3.4, 1.0], [9.0, 2.0, 4.0, -3.8, 6.7]])
b = np.array([5,0,6,3, 8])

for i in range(100):
  start_time = time.time()
  palusolve(A,b)
  PALU_time.append(time.time() - start_time)
  start_time = time.time()
  precond_conj_grad(A,np.zeros_like(b),b)
  conj_grad_time.append(time.time() - start_time)
PALU_time = np.array(PALU_time)
conj_grad_time = np.array(conj_grad_time)
print(np.average(PALU_time))
print(np.average(conj_grad_time))

0.0005291557312011719
0.006359415054321289


In [ ]:
 #computes the derivatives of a function at a given value
def derivatives(f, input, delta):
  derivatives = []
  funcat = f(*input)
  for comp in range(len(input)):
    bump = input.copy()
    bump[comp] += delta
    funcdelt = f(*bump)
    derivatives.append((funcdelt - funcat)/delta)
  return derivatives

def f(x,y):
  return x**2 * y**2 + x*y + 2
#print(derivatives(f,[1,1], 1e-12))

def jacobian(F, delta=1e-12):
  def output(input):
    funcat = F(*input)
    for comp in range(len(input)):
      bump = input.copy()
      bump[comp] += delta
      funcdelt = F(*bump)
      derivatives = (funcdelt - funcat)/delta
      derivatives = derivatives.reshape(-1,1)
      if(comp == 0):
        jaq = derivatives
      else:
        jaq = np.concatenate((jaq,derivatives),axis=1)
    return jaq
  return output

def F(x,y):
  a = x**2 * y
  b = 5*x + np.sin(y)
  return np.array([a,b])
#jacobian should be
#  2xy  x^2
#   5  sin(y)

print(jacobian(F, 1e-12)([3,3]))
#good enough

[[18.00160021  9.00257646]
 [ 5.0004445  -0.98943076]]


In [ ]:
def multiNewton(F, guess, delta=1e-12, maxiter=25000, tol=1e-8, jacfunc=None, chatter=True):
  #use the numerically computed jacobian if no other jacobian is supplied
  if jacfunc == None:
    jacfunc=jacobian(F,delta)

  for iteration in range(maxiter):
    #checks if the F evaluated at the estimated root is within the tolerated distance from 0
    forerror = np.linalg.norm(F(*guess))
    if(forerror <= tol):#essentially forward error
      return guess
    b = F(*guess)
    DF = jacfunc(guess)
    solution = palusolve(DF,-b)
    guess = guess + solution
    #DF s =-F(x_k)
    #x_{k+1} = x_k+ s
  if(chatter):
    print("failed to get a root")
    print(forerror)
  return guess

def F(u,v):
  a = v - u**3
  b = u**2 + v**2 - 1
  return np.array([a,b])

print(multiNewton(F,np.array([1,1])))

[0.82603136 0.56362417]


Our Newton's Method implementation is working correctly. Now all we need to do for part (1) is create our function with our presets, and pass it into Newton's Method.

Note: the implementation of a numerical jacobian above is mostly for completeness, our function is a pretty simple multivariable polynomial and we will be using our own jacobian function computed by symbolic derivation.

In [ ]:
A = [15600, 18760, 17610, 19170]
B = [7540, 2750, 14630, 610]
C = [20140, 18610, 13480, 18390]
t = [0.07074, 0.07220, 0.07690, 0.07242]
c = 299792.458 #kilometers per second

def GPS(x,y,z,d):
  return np.array([(x-A[i])**2 + (y-B[i])**2 + (z-C[i])**2 - (c*(t[i]-d))**2 for i in range(4)])

def jaqGPS(input):
  x = input[0]
  y = input[1]
  z = input[2]
  d = input[3]
  return np.array([[2*(x-A[i]), 2*(y-B[i]), 2*(z-C[i]), 2*c**2 * (t[i]-d)] for i in range(4)])


root = multiNewton(GPS,np.array([0,0,6370,0]), jacfunc=jaqGPS)
for comp in root:
  print(comp)
print(GPS(*root))
print(np.linalg.norm(GPS(*root)))

-41.77270957083659
-16.78919410652534
6370.059559223339
-0.003201565829594143
[0. 0. 0. 0.]
0.0


And our root for the inputs is:


x = -41.77270

y = -16.78919

z = 6370.05955

t = -0.003201

this concludes part 1.

In part 2 we test the conditioning of the system. We have

In [ ]:
from math import pi
rho = 26570.0
phi = pi/2 * np.random.rand(4)
theta = 2 * pi * np.random.rand(4)
A = rho * np.cos(phi) * np.cos(theta)
B = rho * np.cos(phi) * np.sin(theta)
C = rho * np.sin(phi)
print("phi: ", phi)
print("theta: ", theta)
print("A: ", A)
print("B: ", B)
print("C: ", C)

#good starting points to try:
#phi:  [1.46346527 0.57254535 0.41474974 1.02284873]
#theta:  [6.2710199  4.55120411 5.03581859 4.10777656]

#phi:  [1.20518929 1.32700682 0.1446215  1.28451664]
#theta:  [1.29486773 0.98881825 2.75238143 1.96811342]

#phi:  [0.53595228 0.79176088 0.0197512  1.51744748]
#theta:  [6.18706504 2.18136436 1.88256667 3.42450733]

#it seems to break at
#phi:  [1.08170835 1.29146239 1.5464685  1.45836042]
#theta:  [2.03904193 0.75682853 2.50446354 3.2891799 ]

#phi:  [0.60267966 1.08522731 0.68136807 1.03388003]
#theta:  [3.85946342 3.34012104 5.6855919  6.20362733]

#phi:  [0.87795561 0.2426663  1.45030545 0.0701176 ]
#theta:  [0.65486176 4.57491189 3.05571254 3.89472755]

#phi:  [0.25330888 1.40050611 0.7128445  0.97196922]
#theta:  [4.49338061 1.10750605 3.62726127 4.67773982]

phi:  [1.16579824 1.00566523 1.44610105 0.26848605]
theta:  [0.26637235 5.65656699 0.72591326 0.81339062]
A:  [10099.81171062 11525.65744849  2471.46948847 17600.62094084]
B:  [ 2755.79958462 -8343.9769913   2193.63821111 18614.63998225]
C:  [24420.57067432 22438.8539892  26363.70023283  7048.27786258]


In [ ]:
phi = np.array([0.74147186, 0.95692679, 0.36749778, 1.26212913])
theta = np.array([1.63999656, 0.39640358, 4.45769069, 6.19066006])
A = np.array([-1354.87721992, 14118.40823996, -6247.41283722,  8037.15038383])
B = np.array([ 19547.82111963, 5909.40579845, -23995.96984143,   -745.76868836])
C = np.array([17944.68994421, 21718.98643766,  9546.10728047, 25314.28337463])

where the components of phi range from 0 to π/2 and the components of theta range from 0 to 2π. Next we set

In [ ]:
x1=0.0
y1=0.0
z1=6370
d=0.0001

and calculate the radii and travel times:

In [ ]:
R = np.sqrt((A-x1)**2 + (B-y1)**2 + (C-z1)**2 )
original = d + R/c
t=original.copy()
print("radius: ", R)
print("travel time: ", t)
absolute_root = multiNewton(GPS,np.array([0,0,6370,0.0001]), jacfunc=jaqGPS, chatter=False)
print(absolute_root)

radius:  [22757.99749782 21675.83707229 24998.4878192  20592.17885041]
travel time:  [0.07601251 0.07240281 0.08348598 0.06878812]
[ 3.57951825e-12 -1.25003419e-12  6.37000000e+03  1.00000000e-04]


we can recalculate the position just as a sanity check; it should be quite close to our hard coded position

In [ ]:
root = multiNewton(GPS,np.array([0,0,6370,0.0001]), jacfunc=jaqGPS, chatter=False)
for comp in root:
  print(comp)
print(GPS(*root))

jaq = np.linalg.norm(jaqGPS(np.array([0,0,6370,0.0001])))

3.5795182500045474e-12
-1.2500341886533057e-12
6369.9999999999945
0.00010000000000000648
[1.19209290e-07 1.78813934e-07 2.38418579e-07 5.96046448e-08]


Now if we want to know the error magnification factor, we can randomly perturb the time values a little bit

In [ ]:
t = original.copy()
eps = (np.random.randint(0,2,size=4) * 2 - 1) * 10**-8
t = t+eps
print(t)

[0.07289074 0.07109576 0.07053552 0.08702899]


and then recalculate the position

In [ ]:
root = multiNewton(GPS,np.array([0,0,6370,0.0001]), jacfunc=jaqGPS, chatter=False)
for comp in root:
  print(comp)
print(GPS(*root))

-7.745171010502211e+24
1.4570198460422232e+29
3.691385454061655e+28
-3.807005147305079e+22
[2.24614419e+58 2.24614419e+58 2.24614419e+58 2.24614419e+58]


and finally calculate the error factor

In [ ]:
change = np.abs((root-np.array([0,0,6370,0.001])))
maxchange = np.max(change[:-1])
errfactor = maxchange / (c * 10**-8)
print("Deltas: ", change)
print("the maximum error is", errfactor*1000, "meters")

Deltas:  [0.00744627 0.00500023 0.01183698 0.00090002]
the maximum error is 3948.391161424388 meters


it is also useful to simulate this many times, and obtain an approximation for the maximum error factor for a delta_t of 10^-8

In [ ]:
max_errfactor = 0
max_position_change = 0
max_position_error = 0 #overall, euclidean norm
for iter in range(1000):
  t = original.copy()
  eps = (np.random.sample(4) * 2 - 1) * 10**-8
  max_delt_t = np.max(eps)
  t = t+eps
  root = multiNewton(GPS,np.array([0,0,6370,0.0001]), jacfunc=jaqGPS, maxiter=6000, tol=5e-6, chatter=False)
  change = np.abs((root-np.array([0,0,6370,0.001])))
  position_error = np.linalg.norm(root-np.array([0,0,6370,0.001]))#overall change
  #change = np.abs((root-absolute_root))
  #position_error = np.linalg.norm(root-absolute_root)#overall change
  maxchange = np.max(change[:-1])
  errfactor = maxchange/(c*max_delt_t)
  if errfactor > max_errfactor:
    max_errfactor = errfactor
  if maxchange > max_position_change:
    max_position_change = maxchange
  if position_error > max_position_error:
    max_position_error = position_error
print("the maximum error factor is",max_errfactor*1000,"meters, which is our estimation for the condition number of this problem")
print("the maximum positon delta, or change is", max_position_change*1000)
print("the maximum position overall error is", max_position_error*1000)

the maximum error factor is 1948773.761448402 meters, which is our estimation for the condition number of this problem
the maximum positon delta, or change is 14.102970780186297
the maximum position overall error is 17.29676376063042


This concludes part 2. For part 3 we do the similar steps but start with tighter clustered sattelites.

In [ ]:
phi = 0.05 * np.random.rand(4) + .90
theta =  0.05 * np.random.rand(4) + .90
phi = np.array([0.93013452, 0.90241542, 0.92458621, 0.9187206 ])
theta = np.array([0.90375965, 0.92287958, 0.93964964, 0.90026716])#+1
A = rho * np.cos(phi) * np.cos(theta)
B = rho * np.cos(phi) * np.sin(theta)
C = rho * np.sin(phi)
d=0.0001
print("phi: ", phi)
print("theta: ", theta)
print("A: ", A)
print("B: ", B)
print("C: ", C)
R = np.sqrt((A-x1)**2 + (B-y1)**2 + (C-z1)**2 )
original2 = d + R/c
t = original2.copy()
print("radius: ", R)
print("travel time: ", t)
absolute_root2 = multiNewton(GPS,np.array([0,0,6370,0.0001]), jacfunc=jaqGPS, chatter=False)
print(absolute_root2)

phi:  [0.93013452 0.90241542 0.92458621 0.9187206 ]
theta:  [0.90375965 0.92287958 0.93964964 0.90026716]
A:  [ 9825.30920836  9937.58326574  9440.85398733 10019.26173584]
B:  [12477.49963708 13128.93264456 12917.23839028 12632.78475853]
C:  [21301.17841263 20852.82873979 21212.73505135 21118.52370582]
radius:  [21798.27486346 21928.90243161 21824.1049174  21851.58593759]
travel time:  [0.07281122 0.07324694 0.07289738 0.07298904]
[5.35125416e-11 1.43078149e-11 6.37000000e+03 1.00000000e-04]


In [ ]:
max_errfactor = errfactor
max_position_change = maxchange
max_position_error = 0 #overall, euclidean norm
for iter in range(10):
  t = original2.copy()
  eps = (np.random.sample(4) * 2 - 1) * 10**-8
  max_delt_t = np.max(eps)
  t = t+eps
  root = multiNewton(GPS,np.array([0,0,6370,0.0001]), jacfunc=jaqGPS, maxiter=10000, tol=1e-8, chatter=False)
  change = np.abs((root-np.array([0,0,6370,0.001])))
  position_error = np.linalg.norm(root-np.array([0,0,6370,0.001]))#overall change
  #change = np.abs((root-absolute_root2))
  #position_error = np.linalg.norm(root-absolute_root2)#overall change
  maxchange = np.max(change[:-1])
  errfactor = maxchange/(c*max_delt_t)
  if errfactor > max_errfactor:
    max_errfactor = errfactor
  if maxchange > max_position_change:
    max_position_change = maxchange
  if position_error > max_position_error:
    max_position_error = position_error
print("the maximum error factor is",max_errfactor*1000,"meters, which is our estimation for the condition number of this problem")
print("the maximum positon delta, or change is", max_position_change*1000)
print("the maximum position overall error is", max_position_error*1000)

the maximum error factor is 48672009.900470786 meters, which is our estimation for the condition number of this problem
the maximum positon delta, or change is 50292.58673791901
the maximum position overall error is 74589.07658965459


the error factor fluctuates with the values of theta and phi and the conditioning number is computationally somewhat expensive to approximate in this way, but it's abundantly clear that the error factor when the sattelites are closer together is much, much higher.

This concludes part 3.